<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMIN2DOT0_TRAVEL_PO_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install colab-env --quiet
import colab_env

In [ ]:
import google.generativeai as genai
from google.colab import userdata

# Retrieve API key from userdata
GOOGLE_API_KEY = userdata.get('GEMINI')

# Check if API key is retrieved successfully
if GOOGLE_API_KEY is None:
    raise ValueError("API key not found in userdata['GEMINI'].")

# Configure the client with the API key
genai.configure(api_key=GOOGLE_API_KEY)

# List available models and print their names
models = genai.list_models()
for model in models:
    print(model.name)

# Choose the appropriate Gemini 2.0 model from the list above
model = genai.GenerativeModel(model_name='models/gemini-2.0-flash-exp')

print()
# Example 1: Simple text generation with Gemini 2.0
prompt = "Write a short story about a cat who goes on an adventure in a hot air balloon."
response = model.generate_content(contents=[prompt])
print(response.text)

print()

# Example 2: Code generation with Gemini 2.0
prompt = "Write a Python function to calculate the factorial of a number."
response = model.generate_content(
    contents=[prompt],
)
print(response)

In [23]:
import google.generativeai as genai
from google.colab import userdata
import datetime
import calendar
import sqlite3
import json

# Retrieve API key from userdata
GOOGLE_API_KEY = userdata.get('GEMINI')

# Check if API key is retrieved successfully
if GOOGLE_API_KEY is None:
    raise ValueError("API key not found in userdata['GEMINI']. Please set it using `userdata.set('GEMINI', 'YOUR_API_KEY')`.")

# Configure the client with the API key
genai.configure(api_key=GOOGLE_API_KEY)

# Choose the appropriate Gemini 2.0 model
model = genai.GenerativeModel(model_name='models/gemini-2.0-flash-exp')

class MoME:
    """
    A MoME system using SQLite for persistence.
    """
    def __init__(self, db_file="mome.db"):
        self.db_file = db_file
        self.conn = sqlite3.connect(self.db_file)
        self.create_table()

    def create_table(self):
        """Creates the user_data table if it doesn't exist."""
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS user_data (
                user_id TEXT PRIMARY KEY,
                data TEXT
            )
        ''')
        self.conn.commit()

    def get(self, user_id, key=None):
        """Retrieves data from the MoME system."""
        try:
            cursor = self.conn.cursor()
            cursor.execute("SELECT data FROM user_data WHERE user_id=?", (user_id,))
            row = cursor.fetchone()
            if row:
                # Correctly handle potential empty data
                if row[0] is None:
                    data = {}
                else:
                    data = json.loads(row[0])  # Use json.loads for deserialization
                if key:
                    return data.get(key)
                else:
                    return data
            else:
                return None
        except Exception as e:
            print(f"Error retrieving data from MoME: {e}")
            return None

    def save(self, user_id, key, value):
        """Saves data to the MoME system."""
        try:
            data = self.get(user_id)
            if data is None:
                data = {}
            data[key] = value
            cursor = self.conn.cursor()
            cursor.execute("INSERT OR REPLACE INTO user_data (user_id, data) VALUES (?, ?)", (user_id, json.dumps(data)))  # Use json.dumps for serialization
            self.conn.commit()
        except Exception as e:
            print(f"Error saving data to MoME: {e}")


class Agent:
    def __init__(self, model, mome):
        self.model = model
        self.mome = mome

    def get_gemini_response(self, prompt, user_id=None, context=None):
        """Generates a response using Gemini 2.0, incorporating context."""
        if context:
            prompt = f"Conversation history:\n{context}\n\nUser: {prompt}"
        response = self.model.generate_content(contents=[prompt])
        return response.text

    def search_flights(self, origin, destination, month):
        """Searches for flights (simulated)."""
        # Replace with actual flight search API call
        print(f"Searching for flights from {origin} to {destination} in {month} 2025...")
        return [
            {"airline": "Air Canada", "price": 350, "departure_time": "10:00 AM"},
            {"airline": "United", "price": 400, "departure_time": "12:00 PM"},
        ]

    def search_hotels(self, location, month, preferences=None):
        """Searches for hotels (simulated)."""
        # Replace with actual hotel search API call
        print(f"Searching for hotels in {location} in {month} 2025...")
        hotels = [
            {
                "name": "Hotel A",
                "price": 150,
                "rating": 4.5,
                "description": "Close to Central Park",
            },
            {
                "name": "Hotel B",
                "price": 200,
                "rating": 4.2,
                "description": "Near Times Square",
            },
        ]
        if preferences and "hotel_type" in preferences:
            hotels = [
                hotel
                for hotel in hotels
                if preferences["hotel_type"].lower() in hotel["description"].lower()
            ]
        return hotels

    def get_yankees_tickets(self, month):
        """Gets Yankees tickets (simulated)."""
        # Replace with actual ticket purchase API call
        print(f"Getting Yankees tickets for a game in {month} 2025...")
        return {"section": "Legends Suite", "price": 1500}

    def plan_trip(self, user_id, origin, destination, month, budget):
        """Plans a trip using Gemini and MoME."""
        context = ""

        # Get initial response from Gemini
        prompt = f"Plan a trip from {origin} to {destination} in {month} 2025 with a budget of ${budget}, including a Yankees game with Legends Suite seats."
        response = self.get_gemini_response(prompt, user_id, context)
        context += f"User: {prompt}\nAgent: {response}\n"
        print(f"Agent: {response}")

        # Search for flights
        flights = self.search_flights(origin, destination, month)
        flight_options = "\n".join(
            [
                f"{flight['airline']} - ${flight['price']} - {flight['departure_time']}"
                for flight in flights
            ]
        )
        prompt = f"Here are some flight options:\n{flight_options}\nWhich one do you prefer?"
        response = self.get_gemini_response(prompt, user_id, context)
        context += f"User: {prompt}\nAgent: {response}\n"
        print(f"Agent: {response}")

        # Search for hotels (consider preferences from MoME)
        preferences = self.mome.get(user_id, "preferences")
        hotels = self.search_hotels(destination, month, preferences)
        hotel_options = "\n".join(
            [
                f"{hotel['name']} - ${hotel['price']} - {hotel['rating']} stars - {hotel['description']}"
                for hotel in hotels
            ]
        )
        prompt = f"Here are some hotel options:\n{hotel_options}\nWhich one do you prefer?"
        response = self.get_gemini_response(prompt, user_id, context)
        context += f"User: {prompt}\nAgent: {response}\n"
        print(f"Agent: {response}")

        # Get Yankees tickets
        tickets = self.get_yankees_tickets(month)
        prompt = f"I found {tickets['section']} tickets for ${tickets['price']} each. Should I book them?"
        response = self.get_gemini_response(prompt, user_id, context)
        context += f"User: {prompt}\nAgent: {response}\n"
        print(f"Agent: {response}")

        # ... (Continue with other activities and suggestions)

        # Save trip details to MoME
        self.mome.save(user_id, "last_trip", {"destination": destination})

if __name__ == "__main__":
    user_id = "user123"
    origin = "Montreal"
    destination = "New York City"
    month = "May"
    budget = 5000

    # Initialize MoME and Agent (using SQLite)
    mome_system = MoME()
    agent = Agent(model, mome_system)

    agent.plan_trip(user_id, origin, destination, month, budget)

Agent: Okay, let's plan an awesome Montreal to New York City trip in May 2025, with a focus on luxury at a Yankees game with Legends Suite seats, all while staying within a $5000 budget! This will require careful planning and prioritization.

**Trip Overview:**

*   **Duration:** 5 days / 4 nights (adjust to your preference)
*   **Travel:** Train from Montreal to NYC
*   **Accommodation:** Mix of well-located mid-range and potentially a splurge night
*   **Focus:** High-end Yankees experience, exploring key NYC attractions
*   **Budget Breakdown:** (Estimates, will be adjusted based on actual prices)

    *   Transportation: $300
    *   Accommodation: $1500
    *   Yankees Game (Legends Suite): $1500-$2000 (This is the biggest variable)
    *   Food & Drinks: $700
    *   Activities & Attractions: $500
    *   Buffer: $200 - $700 (Always good to have some wiggle room!)

**Itinerary Outline:**

**Day 1: Montreal to NYC & Arrival**

*   **Morning:** Take an early train from Montreal to 